In [21]:
# BMI Multinomial Example

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler      # Normalization
from sklearn.model_selection import train_test_split  # train, test 분리
from sklearn.model_selection import KFold           # cross validation

# Raw Data Loading
df = pd.read_csv('./data/bmi/bmi.csv', skiprows=3)

# 결측치와 이상치 확인 및 처리 (결측치와 이상치가 없어서 이 부분은 스킵)

# Data Split ( 7:3 비율로 데이터를 분리)
x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(df[['height','weight']], df['label'], test_size=0.3, random_state=0)

# x_data_test, t_data_test 두개는 맨 끝에 최종 accuracy를 측정할때 사용된다.

# Normalization
scaler = MinMaxScaler()
scaler.fit(x_data_train)   # 나중에 scaling하기 위한 정보를 scaler에게 세팅
x_data_train_norm = scaler.transform(x_data_train)
x_data_test_norm = scaler.transform(x_data_test)

del x_data_train  # 에러를 방지하기 위해서 사용하지 않는 변수 삭제
del x_data_test

######### Tensorflow 구현 ########

# 정답에 해당하는 t_data_train을 살펴보니 Multinomial이어서 One-How endocing을 이용해서
# 데이터를 변환시켜야 한다.
# 1 0 0 => 0          # one-hot encoding
# 0 1 0 => 1
# 0 0 1 => 2
# 형태로 바꿔줘야 한다.
# Numpy를 이용해서 로직처리하는 방법이나 Tensorflow API를 이용해서 바꾸는 방법이 있다

sess = tf.Session()

t_data_train_onehot = sess.run(tf.one_hot(t_data_train, depth=3))
t_data_test_onehot = sess.run(tf.one_hot(t_data_test, depth=3))

del t_data_train
del t_data_test

# 지금까지 위에서 x_data_train_norm, t_data_train_onehot 만들었다.
# training data set을 준비했다.

# 데이터가 준비되었으니 이제 Tensorflow Graph를 그려보자

# Placeholder
X = tf.placeholder(shape=[None,2], dtype=tf.float32)
T = tf.placeholder(shape=[None,3], dtype=tf.float32)

# Weight & bias
W = tf.Variable(tf.random.normal([2,3]), name='weight')
b = tf.Variable(tf.random.normal([3]), name='bias')

# Hypothesis(Model)
logit = tf.matmul(X,W) + b
H = tf.nn.softmax(logit)    # tf.sigmoid()

# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit,
                                                                 labels=T))

# train 
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)

# 반복학습하는 함수
# parameter
num_of_epoch = 1000
batch_size = 100 # 한번에 학습할 x_data와 t_data의 행의 수

def run_train(sess, train_x, train_t):
    print('###학습이 시작된다 ###')
    sess.run(tf.global_variables_initializer())
    
    total_batch = int(train_x.shape[0] / batch_size)
    
    for step in range(num_of_epoch):
        
        for i in range(total_batch):
            batch_x = train_x[i*batch_size:(i+1)*batch_size]
            batch_t = train_t[i*batch_size:(i+1)*batch_size]
            _, loss_val = sess.run([train, loss], feed_dict={X: x_data_train_norm,
                                                             T: t_data_train_onehot})
        
        if step % 100 == 0:
            print('Loss : {}'.format(loss_val))
    
    print('### 학습이 종료된다.###')
    

# Accuracy(정확도 측정)    #     0    1     2
predict = tf.argmax(H,1)   #  [[0.5  0.4  0.1]] 내가 입력으로 넣은 값에 대한 예측
correct = tf.equal(predict, tf.argmax(T,1))
accuracy = tf.reduce_mean(tf.cast(correct,dtype=tf.float32))

#학습을 진행하고 validation을 수행
# run_train(sess, x_data_train_norm, t_data_train_onehot)
# result = sess.run(accuracy, feed_dict={X: x_data_train_norm,
#                                        T: t_data_train_onehot})
# 이렇게 하는것보다 CrossValidation을 하는게 더 좋다
cv = 2         # [훈련, 검증] => 5 set가 만들어진다.
results = []   # 5 set에 대한 accuracy를 구해서 list안에 차곡차곡 쌓을 거다
kf = KFold(n_splits=cv, shuffle=True)

for training_idx, validation_idx in kf.split(x_data_train_norm):
    # training_idx : 결국은 index값을 알아온다.
    train_x = x_data_train_norm[training_idx]   # Fancy indexing
    train_t = t_data_train_onehot[training_idx]
    
    valid_x = x_data_train_norm[validation_idx]
    valid_t = t_data_train_onehot[validation_idx]
    
    run_train(sess,train_x,train_t)
    results.append(sess.run(accuracy, feed_dict={X: valid_x,
                                                 T: valid_t}))

print('Cross Validation 결과 : {}'.format(results))
print('Cross Validation 최종 결과 : {}'.format(np.mean(results)))
    
    


###학습이 시작된다 ###
Loss : 0.893476128578186
Loss : 0.267341285943985
Loss : 0.20929116010665894
Loss : 0.181229367852211
Loss : 0.16373635828495026
Loss : 0.15144318342208862
Loss : 0.14217166602611542
Loss : 0.1348431557416916
Loss : 0.12885166704654694
Loss : 0.12382805347442627
### 학습이 종료된다.###
###학습이 시작된다 ###
Loss : 0.9175377488136292
Loss : 0.2666444480419159
Loss : 0.20901276171207428
Loss : 0.18106912076473236
Loss : 0.1636282056570053
Loss : 0.15136408805847168
Loss : 0.1421101838350296
Loss : 0.1347935050725937
Loss : 0.12881040573120117
Loss : 0.12379317730665207
### 학습이 종료된다.###
Cross Validation 결과 : [0.9834286, 0.983]
Cross Validation 최종 결과 : 0.983214259147644


In [22]:
result = sess.run(accuracy, feed_dict={X: x_data_test_norm,
                                       T:t_data_test_onehot})

print('최종 accuracy 결과: {}'.format(result))

최종 accuracy 결과: 0.984333336353302


In [23]:
# prediction
height = 187
weight = 78

my_state = [[height,weight]]
my_state_scaled = scaler.transform(my_state)

result = sess.run(H, feed_dict = {X:my_state_scaled})
print(np.argmax(result))

1


In [84]:
# Kaggle

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler      # Normalization
from sklearn.model_selection import train_test_split  # train, test 분리
from sklearn.model_selection import KFold           # cross validation


# Raw Data Loading
train_df = pd.read_csv('./data/digit-recognizer/train.csv')
test_df = pd.read_csv('./data/digit-recognizer/test.csv')

x_data = train_df.drop('label', axis=1, inplace=False).values
t_data = train_df['label'].values

# x_data, x_data_test, t_data, t_data_test = \
# train_test_split(x_data, t_data, test_size=0.75, random_state=0)

# x_data = x_data > 0
# x_data = x_data.astype(int)

print(x_data.shape)
print(x_test.shape)
print(t_data.shape)

# 결측치와 이상치 확인 및 처리 (결측치와 이상치가 없어서 이 부분은 스킵)

# Normalization
scaler = MinMaxScaler()
scaler.fit(x_data)   # 나중에 scaling하기 위한 정보를 scaler에게 세팅
x_data_norm = scaler.transform(x_data)

(42000, 784)
(28000, 784)
(42000,)


In [86]:
# ######### Tensorflow 구현 ########
sess = tf.Session()

# onehot encoding
t_data_onehot = sess.run(tf.one_hot(t_data, depth=10))

# print(t_data_onehot)

# Placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
T = tf.placeholder(shape=[None,10], dtype=tf.float32)

# Weight & bias
W = tf.Variable(tf.zeros([784,10]), name='weight')
b = tf.Variable(tf.zeros([10]), name='bias')

# Hypothesis(Model)
logit = tf.matmul(X,W) + b
H = tf.nn.softmax(logit)    # tf.sigmoid()

# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit,
                                                                 labels=T))

# train 
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)

# 반복학습하는 함수
# parameter
num_of_epoch = 500
batch_size = 200 # 한번에 학습할 x_data와 t_data의 행의 수

def run_train(sess, train_x, train_t):
    print('###학습이 시작된다 ###')
    sess.run(tf.global_variables_initializer())
    
    total_batch = int(train_x.shape[0] / batch_size)
    
    for step in range(num_of_epoch):
        
        for i in range(total_batch):
            batch_x = train_x[i*batch_size:(i+1)*batch_size]
            batch_t = train_t[i*batch_size:(i+1)*batch_size]
            _, loss_val = sess.run([train, loss], feed_dict={X: x_data_norm,
                                                             T: t_data_onehot})
        
        if step % 10 == 0:
            print('Loss : {}'.format(loss_val))
    
    print('### 학습이 종료된다.###')
    
# Accuracy(정확도 측정)
predict = tf.argmax(H,1)
correct = tf.equal(predict, tf.argmax(T,1))
accuracy = tf.reduce_mean(tf.cast(correct,dtype=tf.float32))

#학습을 진행하고 validation을 수행
run_train(sess, x_data_norm, t_data_onehot)
result = sess.run(accuracy, feed_dict={X: x_data_norm,
                                       T: t_data_onehot})

print(result)

# 이렇게 하는것보다 CrossValidation을 하는게 더 좋다
cv = 2         # [훈련, 검증] => 5 set가 만들어진다.
results = []   # 5 set에 대한 accuracy를 구해서 list안에 차곡차곡 쌓을 거다
kf = KFold(n_splits=cv, shuffle=True)

for training_idx, validation_idx in kf.split(x_data_norm):
    # training_idx : 결국은 index값을 알아온다.
    train_x = x_data_norm[training_idx]   # Fancy indexing
    train_t = t_data_onehot[training_idx]
    
    valid_x = x_data_norm[validation_idx]
    valid_t = t_data_onehot[validation_idx]
    
    run_train(sess,train_x,train_t)
    results.append(sess.run(accuracy, feed_dict={X: valid_x,
                                                 T: valid_t}))

print('Cross Validation 결과 : {}'.format(results))
print('Cross Validation 최종 결과 : {}'.format(np.mean(results)))

###학습이 시작된다 ###
Loss : 0.4257556200027466


KeyboardInterrupt: 

In [82]:
test_df = pd.read_csv('./data/digit-recognizer/test.csv')

# test_df = test_df > 0
# test_df = test_df.astype(int)

test_scaled = scaler.transform(test_df)

result = sess.run(H, feed_dict = {X:test_scaled})
result = np.argmax(result, axis=1)
result = pd.Series(result, name='label')

print(result)

submission = pd.read_csv('./data/digit-recognizer/sample_submission.csv')
submission['Label'] = result

submission.to_csv('./submission/digit_200_300_one.csv', index=False)

0        2
1        0
2        9
3        9
4        3
        ..
27995    9
27996    7
27997    3
27998    9
27999    2
Name: label, Length: 28000, dtype: int64
